# Greedy Technique

Given an array of size n that contain either ‘G’rab car or ‘P’assenger. 

Each Grab car can pick up only one passenger. And each Grab car cannot pick up a passenger who is more than K units away from Grab car.

1. Write a program using Brute force approach to find number of all solutions 
    that give the maximum number of Passenger(s) that can ride Grab(s).
2. Write a program using Greedy Technique to find a solution that gives the maximum number of Passengers(s) that can ride Grab(s).

For example, if an array consists of {‘G’, ‘P’, ‘P’, ‘G’, ‘P’} and we set k = 1, 

then the output the maximum number passenger can ride Grab would be 2. 

The first Grab picks up the first passenger and the second Grab picks up either the second or third passenger.

In [27]:
def can_pair(grab_index, passenger_index, k):
    return abs(grab_index - passenger_index) <= k

def cartesian_product(elements, repeat):
    """
    this function helps us to generate all possible combinations of pairings 'G' with passengers

    example:
    if there are 2 'G'rab cars and 3 passengers -> given p_indices = [1,2,4]
    this will generate combinations like [1, 1],[1, 2],[1, 4],[2, 1],[2, 2],[2, 4],[4, 1],[4, 4]
    """
    # Base Case
    if repeat == 0:
        return [[]]
    
    # Recursive Case: Get smaller product
    smaller_product = cartesian_product(elements, repeat-1)
    
    # Build the product using nested loops
    result = []
    for x in elements:
        for suffix in smaller_product:
            combination = [x] + suffix
            result.append(combination)
            
    return result
def brute_force_max_passenger(arr, k):
    g_indices = [i for i, val in enumerate(arr) if val == 'G']
    p_indices = [i for i, val in enumerate(arr) if val == 'P']

    max_pairs = 0
    solutions = []

    for combo in cartesian_product(p_indices, len(g_indices)): 
        is_valid = True
        used = set()
        pairs = []

        for i in range(len(g_indices)):
            if can_pair(g_indices[i], combo[i], k) and combo[i] not in used:
                pairs.append((g_indices[i], combo[i]))
                used.add(combo[i])
            else:
                is_valid = False
                break

        if is_valid and len(pairs) > max_pairs:
            max_pairs = len(pairs)
            solutions = [pairs]
        elif is_valid and len(pairs) == max_pairs:
            solutions.append(pairs)

    return max_pairs, solutions

# Testing
arr = ['G', 'P', 'P', 'G', 'P', 'P','G','G','P','P']
k = 2
max_passengers, solutions = brute_force_max_passenger(arr, k)

print(arr)
print(f"k: {max_passengers}")
for idx, solution in enumerate(solutions, 1):
    print(f"Solution {idx}: {solution}")


['G', 'P', 'P', 'G', 'P', 'P', 'G', 'G', 'P', 'P']
k: 4
Solution 1: [(0, 1), (3, 2), (6, 4), (7, 5)]
Solution 2: [(0, 1), (3, 2), (6, 4), (7, 8)]
Solution 3: [(0, 1), (3, 2), (6, 4), (7, 9)]
Solution 4: [(0, 1), (3, 2), (6, 5), (7, 8)]
Solution 5: [(0, 1), (3, 2), (6, 5), (7, 9)]
Solution 6: [(0, 1), (3, 2), (6, 8), (7, 5)]
Solution 7: [(0, 1), (3, 2), (6, 8), (7, 9)]
Solution 8: [(0, 1), (3, 4), (6, 5), (7, 8)]
Solution 9: [(0, 1), (3, 4), (6, 5), (7, 9)]
Solution 10: [(0, 1), (3, 4), (6, 8), (7, 5)]
Solution 11: [(0, 1), (3, 4), (6, 8), (7, 9)]
Solution 12: [(0, 1), (3, 5), (6, 4), (7, 8)]
Solution 13: [(0, 1), (3, 5), (6, 4), (7, 9)]
Solution 14: [(0, 1), (3, 5), (6, 8), (7, 9)]
Solution 15: [(0, 2), (3, 1), (6, 4), (7, 5)]
Solution 16: [(0, 2), (3, 1), (6, 4), (7, 8)]
Solution 17: [(0, 2), (3, 1), (6, 4), (7, 9)]
Solution 18: [(0, 2), (3, 1), (6, 5), (7, 8)]
Solution 19: [(0, 2), (3, 1), (6, 5), (7, 9)]
Solution 20: [(0, 2), (3, 1), (6, 8), (7, 5)]
Solution 21: [(0, 2), (3, 1), (6,